In [1]:
from mbmvpa.data.loader import BIDSDataLoader

In [2]:
root = "/data2/project_modelbasedMVPA/PRL"
task_name = "prl"
process_name = "absrpe"

subjects = ['01','02','03','04','05','06',
            '07','08','09','10', '11', '12',
            ]

In [3]:
loader = BIDSDataLoader(layout=root, 
                        process_name=process_name, 
                        subjects=subjects,
                        normalizer="minmax")

X_dict,y_dict = loader.get_data(subject_wise=True)
voxel_mask = loader.get_voxel_mask()

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)
subject_01:   0%|          | 0/12 [00:00<?, ?it/s]

INFO: retrieving from BIDS Layout: ...sedMVPA/PRL/derivatives/mbmvpa | Subjects: 47 | Sessions: 47 | Runs: 93
      task-None, process-absrpe
INFO: start loading data


INFO: loaded data info. total-20
      subject_01: X(636, 22192), y(636, 1)
      subject_03: X(636, 22192), y(636, 1)
      subject_04: X(636, 22192), y(636, 1)
      subject_05: X(636, 22192), y(636, 1)
      subject_07: X(636, 22192), y(636, 1)
      subject_08: X(636, 22192), y(636, 1)
      subject_09: X(636, 22192), y(636, 1)
      subject_10: X(636, 22192), y(636, 1)
      subject_11: X(636, 22192), y(636, 1)
      subject_12: X(636, 22192), y(636, 1)
INFO: loaded voxel mask(46, 55, 46)
INFO: loading data done


In [ ]:
from mbmvpa.models.mvpa_general import MVPA_CV
from mbmvpa.models.elasticnet import MVPA_ElasticNet
from mbmvpa.utils.report import build_elasticnet_report_functions

report_path = "ccsl_prl_tutorial"
Path(report_path).mkdir(exist_ok=True)

In [ ]:
model = MVPA_ElasticNet(alpha=0.01,
                         n_samples=100000,
                         shuffle=True,
                         max_lambda=50,
                         min_lambda_ratio=1e-4,
                         lambda_search_num=100,
                         n_jobs=16,
                         n_splits=5)


In [ ]:
report_function_dict = build_elasticnet_report_functions(voxel_mask,
                                                         confidence_interval=.99,
                                                         n_coef_plot=150,
                                                         task_name=task_name+"-"+process_name,
                                                         map_type='z',
                                                         sigma=1
                                                         )

In [ ]:
model_cv = MVPA_CV(X_dict,
                    y_dict,
                    model,
                    model_param_dict={},
                    method='5-fold',
                    n_cv_repeat=5,
                    cv_save=True,
                    cv_save_path=report_path,
                    task_name=task_name,
                    report_function_dict=report_function_dict)


In [ ]:
model_cv.run()